In [9]:
import pandas as pd
import numpy as np
import scipy.stats as scipy
from __future__ import division

In [10]:
df = pd.read_csv("./Data/England/E0_13.csv")
df_14 = pd.read_csv("./Data/England/E0_14.csv")

In [11]:
df.columns

Index([u'Div', u'Date', u'HomeTeam', u'AwayTeam', u'FTHG', u'FTAG', u'FTR',
       u'HTHG', u'HTAG', u'HTR', u'Referee', u'HS', u'AS', u'HST', u'AST',
       u'HF', u'AF', u'HC', u'AC', u'HY', u'AY', u'HR', u'AR', u'B365H',
       u'B365D', u'B365A', u'BWH', u'BWD', u'BWA', u'IWH', u'IWD', u'IWA',
       u'LBH', u'LBD', u'LBA', u'PSH', u'PSD', u'PSA', u'WHH', u'WHD', u'WHA',
       u'SJH', u'SJD', u'SJA', u'VCH', u'VCD', u'VCA', u'Bb1X2', u'BbMxH',
       u'BbAvH', u'BbMxD', u'BbAvD', u'BbMxA', u'BbAvA', u'BbOU', u'BbMx>2.5',
       u'BbAv>2.5', u'BbMx<2.5', u'BbAv<2.5', u'BbAH', u'BbAHh', u'BbMxAHH',
       u'BbAvAHH', u'BbMxAHA', u'BbAvAHA', u'PSCH', u'PSCD', u'PSCA'],
      dtype='object')

In [12]:
res_13 = df.ix[:,:23]
res_13 = res_13.drop(['Div','Date','Referee'],axis=1)
res_14 = df_14.ix[:,:23]
res_14 = res_14.drop(['Div','Date','Referee'],axis=1)
table_features = df.ix[:,:7]
table_features = table_features.drop(['FTHG','FTAG','Div','Date'],axis=1)
bet_13 = df.ix[:,23:]


In [6]:
res_13.head()
feature_table = df.ix[:,:23]

In [7]:
#Team, Home Goals Score, Away Goals Score, Attack Strength, Home Goals Conceded, Away Goals Conceded, Defensive Strength
table_13 = pd.DataFrame(columns=('Team','HGS','AGS','HAS','AAS','HGC','AGC','HDS','ADS',))

In [8]:
avg_home_scored_13 = res_13.FTHG.sum() / 380.0
avg_away_scored_13 = res_13.FTAG.sum() / 380.0
avg_home_conceded_13 = avg_away_scored_13
avg_away_conceded_13 = avg_home_scored_13
print "Average number of goals at home",avg_home_scored_13
print "Average number of goals away", avg_away_scored_13
print "Average number of goals conceded at home",avg_away_conceded_13
print "Average number of goals conceded away",avg_home_conceded_13


Average number of goals at home 1.57368421053
Average number of goals away 1.19473684211
Average number of goals conceded at home 1.57368421053
Average number of goals conceded away 1.19473684211


In [9]:
res_home = res_13.groupby('HomeTeam')
res_away = res_13.groupby('AwayTeam')

In [10]:
# list(res_home)[0]

In [11]:
table_13.Team = res_home.HomeTeam.all().values
table_13.HGS = res_home.FTHG.sum().values
table_13.HGC = res_home.FTAG.sum().values
table_13.AGS = res_away.FTAG.sum().values
table_13.AGC = res_away.FTHG.sum().values
table_13.head()

,Team,HGS,AGS,HAS,AAS,HGC,AGC,HDS,ADS
0,Arsenal,36,32,NaN,NaN,11,30,NaN,NaN
1,Aston Villa,22,17,NaN,NaN,29,32,NaN,NaN
2,Cardiff,20,12,NaN,NaN,35,39,NaN,NaN
3,Chelsea,43,28,NaN,NaN,11,16,NaN,NaN
4,Crystal Palace,18,15,NaN,NaN,23,25,NaN,NaN


In [12]:
table_13.HAS = (table_13.HGS / 19.0) / avg_home_scored_13
table_13.AAS = (table_13.AGS / 19.0) / avg_away_scored_13
table_13.HDS = (table_13.HGC / 19.0) / avg_home_conceded_13
table_13.ADS = (table_13.AGC / 19.0) / avg_away_conceded_13
table_13.head()

,Team,HGS,AGS,HAS,AAS,HGC,AGC,HDS,ADS
0,Arsenal,36,32,1.204013,1.409692,11,30,0.484581,1.003344
1,Aston Villa,22,17,0.735786,0.748899,29,32,1.277533,1.070234
2,Cardiff,20,12,0.668896,0.528634,35,39,1.541850,1.304348
3,Chelsea,43,28,1.438127,1.233480,11,16,0.484581,0.535117
4,Crystal Palace,18,15,0.602007,0.660793,23,25,1.013216,0.836120


In [13]:
feature_table = feature_table[['HomeTeam','AwayTeam','FTR','HST','AST']]
f_HAS = []
f_HDS = []
f_AAS = []
f_ADS = []
for index,row in feature_table.iterrows():
    f_HAS.append(table_13[table_13['Team'] == row['HomeTeam']]['HAS'].values[0])
    f_HDS.append(table_13[table_13['Team'] == row['HomeTeam']]['HDS'].values[0])
    f_AAS.append(table_13[table_13['Team'] == row['HomeTeam']]['AAS'].values[0])
    f_ADS.append(table_13[table_13['Team'] == row['HomeTeam']]['ADS'].values[0])
    
feature_table['HAS'] = f_HAS
feature_table['HDS'] = f_HDS
feature_table['AAS'] = f_AAS
feature_table['ADS'] = f_ADS

In [14]:
feature_table.head()

,HomeTeam,AwayTeam,FTR,HST,AST,HAS,HDS,AAS,ADS
0,Arsenal,Aston Villa,A,4,4,1.204013,0.484581,1.409692,1.003344
1,Liverpool,Stoke,H,11,4,1.772575,0.792952,2.114537,1.070234
2,Norwich,Everton,D,2,6,0.568562,0.792952,0.484581,1.471572
3,Sunderland,Fulham,A,3,1,0.702341,1.189427,0.881057,1.103679
4,Swansea,Man United,A,6,7,1.103679,1.145374,0.925110,0.936455


In [15]:
def transformResult(row):
    if(row.FTR == 'H'):
        return 1
    elif(row.FTR == 'A'):
        return -1
    else:
        return 0

In [16]:
feature_table["Result"] = feature_table.apply(lambda row: transformResult(row),axis=1)

In [17]:
feature_table.head()

,HomeTeam,AwayTeam,FTR,HST,AST,HAS,HDS,AAS,ADS,Result
0,Arsenal,Aston Villa,A,4,4,1.204013,0.484581,1.409692,1.003344,-1
1,Liverpool,Stoke,H,11,4,1.772575,0.792952,2.114537,1.070234,1
2,Norwich,Everton,D,2,6,0.568562,0.792952,0.484581,1.471572,0
3,Sunderland,Fulham,A,3,1,0.702341,1.189427,0.881057,1.103679,-1
4,Swansea,Man United,A,6,7,1.103679,1.145374,0.925110,0.936455,-1


In [18]:
X_train = feature_table[['HST','AST','HAS','HDS','AAS','ADS',]]
y_train = feature_table['Result']

In [297]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [298]:
clf = DecisionTreeClassifier()
clf2 = MultinomialNB()
clf3 = XGBClassifier()
clf3.fit(X_train,y_train)

XGBClassifier(base_score=0.5, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, nthread=-1, objective='multi:softprob', seed=0,
       silent=True, subsample=1)

In [299]:
y_pred = clf3.predict(X_train)
accuracy_score(y_pred,y_train)
scores = cross_val_score(clf2, X_train, y_train, cv=10)


In [300]:
print scores
print scores.mean()

[ 0.51282051  0.41025641  0.48717949  0.39473684  0.55263158  0.57894737
  0.52631579  0.5         0.48648649  0.47222222]
0.492159669791


## Adding Recent Performance

In [189]:
ht = res_13.loc[40].HomeTeam
at = res_13.loc[40].AwayTeam

In [190]:
feat_table = res_13.sort_index(ascending=False)
feat_table = feat_table[['HomeTeam','AwayTeam','FTR','FTHG','FTAG','HS','AS','HC','AC']]
feat_table["pastHS"] = 0.0
feat_table["pastHC"] = 0.0
feat_table["pastAS"] = 0.0
feat_table["pastAC"] = 0.0
feat_table["pastHG"] = 0.0
feat_table["pastAG"] = 0.0



In [191]:
feat_table.head()

,HomeTeam,AwayTeam,FTR,FTHG,FTAG,HS,AS,HC,AC,pastHS,pastHC,pastAS,pastAC,pastHG,pastAG
379,West Brom,Stoke,A,1,2,17,15,11,6,0.0,0.0,0.0,0.0,0.0,0.0
378,Tottenham,Aston Villa,H,3,0,12,4,5,1,0.0,0.0,0.0,0.0,0.0,0.0
377,Sunderland,Swansea,A,1,3,20,8,6,3,0.0,0.0,0.0,0.0,0.0,0.0
376,Southampton,Man United,D,1,1,15,8,6,2,0.0,0.0,0.0,0.0,0.0,0.0
375,Norwich,Arsenal,A,0,2,11,14,4,4,0.0,0.0,0.0,0.0,0.0,0.0


In [321]:
k = 3
for i in range(379,-1,-1):
    row = feat_table.loc[i]
    ht = row.HomeTeam
    at = row.AwayTeam
    ht_stats = feat_table.loc[i-1:-1][(feat_table.HomeTeam == ht) | (feat_table.AwayTeam == ht)].head(k)
    at_stats = feat_table.loc[i-1:-1][(feat_table.HomeTeam == at) | (feat_table.AwayTeam == at)].head(k)

    feat_table.set_value(i, 'pastHC', (ht_stats[ht_stats["AwayTeam"] == ht].sum().HC + ht_stats[ht_stats["HomeTeam"] == ht].sum().HC)/k)
    feat_table.set_value(i, 'pastAC', (at_stats[at_stats["AwayTeam"] == at].sum().HC + at_stats[at_stats["HomeTeam"] == at].sum().HC)/k)
    feat_table.set_value(i, 'pastHS', (ht_stats[ht_stats["AwayTeam"] == ht].sum().HS + ht_stats[ht_stats["HomeTeam"] == ht].sum().AS)/k)
    feat_table.set_value(i, 'pastAS', (at_stats[at_stats["AwayTeam"] == at].sum().HS + at_stats[at_stats["HomeTeam"] == at].sum().AS)/k)
    feat_table.set_value(i, 'pastHG', (ht_stats[ht_stats["AwayTeam"] == ht].sum().FTAG + ht_stats[ht_stats["HomeTeam"] == ht].sum().FTHG)/k)
    feat_table.set_value(i, 'pastAG', (at_stats[at_stats["AwayTeam"] == at].sum().FTAG + at_stats[at_stats["HomeTeam"] == at].sum().FTHG)/k)

f_HAS = []
f_HDS = []
f_AAS = []
f_ADS = []
for index,row in feature_table.iterrows():
    f_HAS.append(table_13[table_13['Team'] == row['HomeTeam']]['HAS'].values[0])
    f_HDS.append(table_13[table_13['Team'] == row['HomeTeam']]['HDS'].values[0])
    f_AAS.append(table_13[table_13['Team'] == row['HomeTeam']]['AAS'].values[0])
    f_ADS.append(table_13[table_13['Team'] == row['HomeTeam']]['ADS'].values[0])
    
feat_table['HAS'] = f_HAS
feat_table['HDS'] = f_HDS
feat_table['AAS'] = f_AAS
feat_table['ADS'] = f_ADS

/home/sibi/acad/prog_tools/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/sibi/acad/prog_tools/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [322]:
test_table = feat_table.drop(['FTHG','FTAG','HS','AS','HC','AC'],axis=1)

In [323]:
test_table["Result"] = test_table.apply(lambda row: transformResult(row),axis=1)
test_table.sort_index().head()

,HomeTeam,AwayTeam,FTR,pastHS,pastHC,pastAS,pastAC,pastHG,pastAG,HAS,HDS,AAS,ADS,Result
0,Arsenal,Aston Villa,A,0.0,0.0,0.0,0.0,0.0,0.0,0.802676,1.189427,0.837004,1.070234,-1
1,Liverpool,Stoke,H,0.0,0.0,0.0,0.0,0.0,0.0,1.003344,1.013216,1.101322,0.936455,1
2,Norwich,Everton,D,0.0,0.0,0.0,0.0,0.0,0.0,0.702341,1.189427,0.881057,1.103679,0
3,Sunderland,Fulham,A,0.0,0.0,0.0,0.0,0.0,0.0,1.070234,1.013216,0.969163,0.769231,-1
4,Swansea,Man United,A,0.0,0.0,0.0,0.0,0.0,0.0,0.568562,0.792952,0.484581,1.471572,-1


In [13]:
X_train = test_table[['pastHS','pastHC','pastAS','pastAC','pastHG','pastAG','HAS','HDS','AAS','ADS']]
y_train = test_table['Result']

NameError: name 'test_table' is not defined

In [325]:
clf1 = DecisionTreeClassifier()
clf2 = MultinomialNB(alpha=3)
clf3 = XGBClassifier()
clf4 = RandomForestClassifier()
clf5 = KNeighborsClassifier(n_neighbors=5)
clf5.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [326]:
accuracy_score(y_train,clf5.predict(X_train))

0.59999999999999998

In [342]:
scores = cross_val_score(clf2, X_train, y_train, cv=10)
print scores, scores.mean()

[ 0.46153846  0.43589744  0.48717949  0.42105263  0.5         0.55263158
  0.52631579  0.5         0.51351351  0.44444444] 0.484257334257


In [330]:
for n in xrange(1,30):
    clf5 = KNeighborsClassifier(n_neighbors=n)
    scores = cross_val_score(clf5, X_train, y_train, cv=10)
    print scores.mean()


0.400661390135
0.379211498948
0.421376943745
0.415889168521
0.429687784951
0.439734167366
0.434759455812
0.421526789948
0.429496601865
0.416046309467
0.426366548735
0.424361101993
0.434819941399
0.432600515495
0.429833983781
0.416597366597
0.429687784951
0.442721972985
0.432604162867
0.437799844379
0.437653645548
0.429901460165
0.429627907259
0.437593767857
0.432398086345
0.424570825887
0.429837631153
0.432255534887
0.440153919101
